In [ ]:
import enoki as ek
import mitsuba
mitsuba.set_variant("packet_rgb")

import pycode.BRDF as pyxBRDF
from pycode.mitsuba_ext import Frame

from mitsuba.core import Float, Vector3f

import numpy as np

def microfacetBRDF(linRough, wo, wi):
    wh = wo + wi
    wh = ek.normalize(wh)
    ggxAlpha = linRough**2

    cosThetaH = Frame.cos_theta(wh)
    cosThetaI = Frame.cos_theta(wi)
    cosThetaO = Frame.cos_theta(wo)

    D = pyxBRDF.NDF_GGX(ggxAlpha, cosThetaH)
    G = pyxBRDF.G_GGX(ggxAlpha, cosThetaO, cosThetaI)
    return D * G / (4 * cosThetaO)


def ggx_dominant_direction_exact(wo, linRough):
    num_samples = 2048

    theta = ek.linspace(Float, -np.pi/2, np.pi/2, num_samples)
    phi = Float(0.0)
    wi = sph.spherical_dir(theta, phi)

    brdf = microfacetBRDF(linRough, wo, wi)
    ind = np.argmax(brdf.numpy(), axis=None)
    return np.array([wi.x[ind], wi.y[ind] * 0, wi.z[ind]])


def ggx_dominant_direction_frostbite(wo, linRough):
    R = np.array([-wo[0], wo[1], wo[2]])
    N = np.array([0, 0, 1])

    ggxAlpha = linRough**2
    smoothness = 1 - ggxAlpha;
    factor = smoothness * (np.sqrt(smoothness) + ggxAlpha);
    return N * (1 - factor) + R * factor


def ggx_dominant_direction_filament(wo, linRough):
    R = np.array([-wo[0], wo[1], wo[2]])
    N = np.array([0, 0, 1])

    ggxAlpha = linRough**2
    a2 = ggxAlpha**2
    return R * (1 - a2) + N * a2

In [ ]:
import plotly.graph_objects as go
import pycode.spherical as sph
import numpy as np

num_samples = 256
linearRoughness = 0.75
theta_o = 75.0/180*ek.pi


theta, phi = sph.meshgrid_spherical(num_samples, num_samples, hemisphere=True)
wi = sph.spherical_dir(theta, phi)
wo = sph.spherical_dir(theta_o, 0)

vals = microfacetBRDF(linearRoughness, wo, wi)

x, y, z = [comp.numpy().reshape(num_samples, num_samples) for comp in [wi.x, wi.y, wi.z]]
vals_np = vals.numpy().reshape(num_samples, num_samples)

fig = go.Figure()
fig.add_trace(go.Surface(x=x*vals_np, y=y*vals_np, z=z*vals_np, surfacecolor=vals_np))

max_val = np.max(vals_np) * 1.1
fig.update_layout(scene=dict(xaxis=dict(range=[-max_val,max_val]),
                             yaxis=dict(range=[-max_val,max_val]),
                             zaxis=dict(range=[-max_val,max_val]),
                             aspectmode='manual',
                             aspectratio=dict(x=1, y=1, z=1)))


def plot_line(p, color):
    fig.add_trace(go.Scatter3d(x=[0, p[0]], y=[0, p[1]], z=[0, p[2]],
                               mode='lines',
                               showlegend=False,
                               line=dict(color=color)))


def brdf_dominant_direction(brdf_vals, wi):
    ind = np.argmax(brdf_vals, axis=None)
    return np.array([wi.x[ind], wi.y[ind] * 0, wi.z[ind]])


plot_line(wo.numpy()[0] * 100, 'red')
plot_line(wo.numpy()[0] * [-1, 1, 1] * 100, 'red')
# plot_line(brdf_dominant_direction(vals_np, wi) * 100, 'green')
plot_line(ggx_dominant_direction_exact(wo, linearRoughness) * 100, 'gray')
plot_line(ggx_dominant_direction_frostbite(wo.numpy()[0], linearRoughness) * 100, 'blue')
plot_line(ggx_dominant_direction_filament(wo.numpy()[0], linearRoughness) * 100, 'aqua')

fig.show()


In [ ]:
def interpolation_factor(R, N, direction):
    R = R / np.linalg.norm(R)
    N = N / np.linalg.norm(N)
    R = R.reshape(-1, 1)
    N = N.reshape(-1, 1)

    delta = 0.001
    factor = np.arange(0.0, 1.0 + delta, delta).reshape(1, -1)
    V = R * (1 - factor) + N * factor
    V = V.transpose()
    V /= np.linalg.norm(V, axis=1).reshape(-1, 1)

    dist = np.dot(V, direction)
    ind = np.argmax(dist, axis=None)
    return factor[0][ind]


R = wo.numpy()[0] * [-1, 1, 1]
N = np.array([0, 0, 1])
dominant_dir = ggx_dominant_direction_exact(wo, linearRoughness)
tmp = interpolation_factor(R, N, dominant_dir)
dominant_dir_ = R * (1 - tmp) + N * tmp
dominant_dir_ /= np.linalg.norm(dominant_dir_)
print(dominant_dir)
print(dominant_dir_)

In [ ]:
res = 32
lut = np.zeros((res, res))
for i in range(res):
    linearRoughness = (i + 0.5) / res

    for j in range(res):
        theta_o = (j + 0.5) / res * 0.5 * np.pi

        wo = sph.spherical_dir(theta_o, 0)
        R = wo.numpy()[0] * [-1, 1, 1]
        N = np.array([0, 0, 1])

        dominant_dir = ggx_dominant_direction_exact(wo, linearRoughness)
        interp = interpolation_factor(R, N, dominant_dir)
        dominant_dir_ = R * (1-interp) + N * interp
        dominant_dir_ /= np.linalg.norm(dominant_dir_)

        lut[j][i] = interp


In [ ]:
def interp_factor_frostbite(linRough, NdotV):
    ggxAlpha = linRough**2

    # best fit
    lerpFactor = 0.298475 * NdotV * np.log(39.4115 - 39.0029 * ggxAlpha) + (0.385503 -0.385503 * NdotV) * np.log(13.1567 - 12.2848 * ggxAlpha)
    return 1 - lerpFactor

    # coarse fit
    smoothness = 1 - ggxAlpha;
    factor = smoothness * (np.sqrt(smoothness) + ggxAlpha);
    return 1 - factor


def interp_factor_filament(linRough):
    ggxAlpha = linRough**2
    a2 = ggxAlpha**2
    return a2

rough = (np.arange(0, res) + 0.5) / res
theta_o = (np.arange(0, res) + 0.5) / res * 0.5 * np.pi
rough, theta_o = np.meshgrid(rough, theta_o)
fig = go.Figure()
fig.add_trace(go.Surface(x=rough, y=theta_o, z=lut - interp_factor_filament(rough)))
# fig.add_trace(go.Surface(x=rough, y=theta_o, z=interp_factor_filament(rough)))
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Surface(x=rough, y=theta_o, z=lut - interp_factor_frostbite(rough, np.cos(theta_o))))
# fig.add_trace(go.Surface(x=rough, y=theta_o, z=interp_factor_frostbite(rough)))
# fig.add_trace(go.Surface(x=rough, y=theta_o, z=y**2))
fig.show()
